# PyTorch LSTM Overview

Source: https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

LSTMs in Pytorch:
- Pytorch’s LSTM expects all of its inputs to be 3D tensors
- The semantics of the axes of these tensors is important. 
- The first axis is the sequence itself, the second indexes instances in the mini-batch, and the third indexes elements of the input. 
- We haven’t discussed mini-batching, so let’s just ignore that and assume we will always have just 1 dimension on the second axis

Example: For the prompt "The cow jumped" can be represented as

![row vector](row_vector.png)


Remember, this is for mini-batch size 1

In [1]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm import tqdm

torch.manual_seed(1)

In [2]:
lstm = nn.LSTM(3, 3) # input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)] # make a sequence of length 5

print(inputs)

[tensor([[-0.5525,  0.6355, -0.3968]]), tensor([[-0.6571, -1.6428,  0.9803]]), tensor([[-0.0421, -0.8206,  0.3133]]), tensor([[-1.1352,  0.3773, -0.2824]]), tensor([[-2.5667, -1.4303,  0.5009]])]


We can input each sequence at a time where at each step, hidden contains the hidden state.
- The first position is the short term hidden state
- The second position is the long term cell state

In [3]:
# initialize the hidden state
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    
    # step through the sequence one element at a time
    # after each step, hidden contains the hidden state
    out, hidden = lstm(i.view(1, 1, -1), hidden)

In [4]:
print("input sequence on element at a time \n")
print(f"last out: {out} \n")
print(f"last hidden: {hidden}")

input sequence on element at a time 

last out: tensor([[[-0.3600,  0.0893,  0.0215]]], grad_fn=<MkldnnRnnLayerBackward0>) 

last hidden: (tensor([[[-0.3600,  0.0893,  0.0215]]], grad_fn=<StackBackward0>), tensor([[[-1.1298,  0.4467,  0.0254]]], grad_fn=<StackBackward0>))


Alternatively, we can also input the entire sequence at one. 
- The first value returned by the LSTM, `out`, is all of the hidden states throughout the sequence
- The second output, `hidden`, is just the most recent hidden state

In [5]:
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))
out, hidden = lstm(inputs, hidden)

In [6]:
print("input full sequence at once \n")
print(f"last out: {out} \n")
print(f"last hidden: {hidden}")

input full sequence at once 

last out: tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<MkldnnRnnLayerBackward0>) 

last hidden: (tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward0>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward0>))


# Example from PyTorch

An LSTM for Part-of-Speech Tagging
- In this section, we will use an LSTM to get part of speech tags
- Look below for more information about this example

![part-of-speech](part_of_speech_tagging.png)

Prepare our data:

In [7]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

training_data = [
    # Targs are: DET - determiner; NN - noun; V - verb
    # For example, the word "The" is a determiner
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}

# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        
        if word not in word_to_ix: # this check if the word has not beed assigned an index yet
            word_to_ix[word] = len(word_to_ix) # this will assign each word with a unique index
            
print(word_to_ix)

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [8]:
tag_to_ix = {"DET": 0, "NN": 1, "V": 2} # assign each tag to a unique index as well

# these will usually be more like 32 or 64 dimensional
# we will keep them small, so we can see how the weights change as we train
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

Create the model:

In [9]:
class LSTMTagger(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        # the LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        # the linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        
    def forward(self, sentence):
        # convert the sentence into embeddings
        embeds = self.word_embeddings(sentence)
        
        # feed the embeddings into the lstm model
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        
        # map the hidden state space to tage space
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        
        # get tag scores from log softmax
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

Train the model:

In [10]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# see what the scores are before training
# note that the element i,j of the output is the score for tag j for word i
# here we dont nee to train so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

tensor([[-1.1389, -1.2024, -0.9693],
        [-1.1065, -1.2200, -0.9834],
        [-1.1286, -1.2093, -0.9726],
        [-1.1190, -1.1960, -0.9916],
        [-1.0137, -1.2642, -1.0366]])


In [11]:
for epoch in tqdm(range(300)): # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # step 1: remember that pytorch accumulates gradients
        # we need to clear them out before each instance
        model.zero_grad()
        
        # step 2: get inputs ready for the network
        # turn them into tensors of word indices
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        # step 3: run forward pass
        tag_scores = model(sentence_in)
        
        # step 4: compute the loss, gradients and update parameters
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

100%|██████████| 300/300 [00:01<00:00, 212.60it/s]


In [12]:
print(f"Loss: {loss}")

Loss: 0.03095390647649765


In [13]:
# see scores after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    
    # the sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-0.0462, -4.0106, -3.6096],
        [-4.8205, -0.0286, -3.9045],
        [-3.7876, -4.1355, -0.0394],
        [-0.0185, -4.7874, -4.6013],
        [-5.7881, -0.0186, -4.1778]])
